<a href="https://www.kaggle.com/code/hydramst/water-pollution-linreg?scriptVersionId=132380010" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt 
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import r2_score
from sklearn.linear_model import Ridge
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_validate
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNetCV
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import GridSearchCV

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
DF_train = pd.read_csv('/kaggle/input/water-pollution/water_pollution_dataset/Train.csv', delimiter = ',')
DF_test = pd.read_csv('/kaggle/input/water-pollution/water_pollution_dataset/Test.csv', delimiter = ',')
Target = pd.read_csv('/kaggle/input/water-pollution/water_pollution_dataset/Target.csv', delimiter = ',')
Submission = pd.read_csv('/kaggle/input/water-pollution/water_pollution_dataset/Submission.csv', delimiter = ',')

In [ ]:
Target

# Check if ID's in train presented in train

In [ ]:
print('intersection of train and test ID\'s:', 
      len(set(DF_train['id'].values).intersection(set(DF_test['id'].values)))/len(set(DF_test['id'].values)))

63% of test ID's presented in train

In [ ]:
ids_intest = pd.DataFrame(DF_test['id'].values)
ids_intrain = pd.DataFrame(set(DF_train['id'].values).intersection(set(DF_test['id'].values)))

In [ ]:
in_train = []
notin_train = []

for value in ids_intest.values:
    if (value in ids_intrain.values):
        in_train +=[value]
    else:
        notin_train +=[value]
        
id_notin_train = pd.DataFrame(notin_train, columns=['id'])
id_in_train = pd.DataFrame(in_train, columns=['id'])

print('ID presented in train:',len(in_train))

print( 'ID not presented in train:', len(notin_train))

### Concatinate train and test

In [ ]:
df = pd.concat([DF_train, DF_test])

In [ ]:
df = df.drop(columns = ['Unnamed: 0'])

## View categorical and numerical columns

In [ ]:
cat_columns = []
num_columns = []

for column_name in df.columns:
    if (df[column_name].dtypes == object):
        cat_columns +=[column_name]
    else:
        num_columns +=[column_name]

print('categorical columns:\t ',cat_columns, '\n len = ',len(cat_columns))

print('numerical columns:\t ',  num_columns, '\n len = ',len(num_columns))

## Move 'year' to num columns 

In [ ]:
cat_columns = ['code', 'period', 'id', 'Country', 'year']
num_columns = [ 'tourists', 'venue', 'rate', 'food', 'glass', 'metal', 'other', 'paper', 'plastic', 'leather', 'green_waste', 'waste_recycling'] 

### Fill NaN values with mean and ffill (for object) values

In [ ]:
df_new = pd.concat([df, id_notin_train])

In [ ]:
df_new[num_columns].fillna(df[num_columns].mean())
df_new = df.ffill()
df_new.info()

### Fill missing Target values with mean

In [ ]:
mean_targ = Target['polution'].mean()
targ = []
for i in range(0, len(id_notin_train)):
    targ += [mean_targ]
    
#Target = pd.concat([Target, pd.DataFrame(targ, columns=['polution'])])

## OHE 

In [ ]:
df_ohe = pd.get_dummies(df[cat_columns])
df_all = pd.concat([df_ohe, df[num_columns]], axis=1)

# Scaling

In [ ]:
# Creating StandardScaler Object
scaler = StandardScaler()
df_scale = pd.DataFrame(scaler.fit_transform(df_all[num_columns]))
df_all = pd.concat([df_ohe.reset_index(drop=True), df_scale.reset_index(drop=True)], axis=1)

## Split back to test and train 


In [ ]:
len_test = len(DF_test)
train = df_all.iloc[:-len_test]
test = df_all.iloc[-len_test:]
print(DF_train.shape[0], train.shape[0], DF_test.shape[0], test.shape[0])

# Log Scaling of Target

In [ ]:
target_log = np.log(Target['polution'])
target_orig = Target['polution']

# Traintest split

In [ ]:
X, y = pd.DataFrame(train).values, target_orig.values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train.shape[0], X_test.shape[0], y_train.shape[0], y_test.shape[0]

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

# GridSearchCV

In [ ]:
### define model
model = Ridge()

alphas = np.array([1,0.1,0.01,0.001,0.0001,0.000001, 0.0000001])

# define parameters
param = {
    'alpha':alphas,
    'positive':[True, False],
    'copy_X':[True,False],
    'fit_intercept':[True,False]
    
}
#grid = GridSearchCV(model, param, scoring='r2', cv=2, n_jobs=-1)
#grid.fit(X_train, y_train)
#predictions = grid.predict(X_test)
#print('R2 : %.4f' %  r2_score(y_test,predictions))
#print('R2 : %.4f' % grid.best_score_)

# ElasticNetCV

In [ ]:
model_ElasticCV = ElasticNetCV(n_alphas=1, fit_intercept=True, max_iter=100000, l1_ratio=0.9, eps=0.0001, cv=5)

#model_ElasticCV.fit(X_train, y_train)

#y_predict_ElasticCV = model_ElasticCV.predict(X_test)

#print('R2 : %.4f' %  r2_score(y_test,y_predict_ElasticCV))

# ElasticNet

In [ ]:
model_Elastic = ElasticNet(alpha=0.005, fit_intercept=True, max_iter=100000, l1_ratio=0.1)

model_Elastic.fit(X_train, y_train)

y_predict_Elastic = model_Elastic.predict(X_test)

print('R2 : %.4f' %  r2_score(y_test,y_predict_Elastic))

# Ridge LR

In [ ]:
model = Ridge(alpha = 0.1, fit_intercept=True, max_iter=10000)

model.fit(X_train, y_train)

y_predict=model.predict(X_test)

print('Error on test data:')
print('MSE: %.1f' % mse(y_test,y_predict))
print('RMSE: %.1f' % mse(y_test,y_predict,squared=False))
print('R2 : %.4f' %  r2_score(y_test,y_predict))

# Lasso

In [ ]:
model_L = Lasso(alpha=0.00001, fit_intercept=True)

model_L.fit(X_train, y_train)
y_predict=model_L.predict(X_test)

print('R2 : %.4f' %  r2_score(y_test,y_predict))

## Compare answers with original values

In [ ]:
def plotGraph(y_test,y_predict,regressorName):
    if max(y_test) >= max(y_predict):
        my_range = int(max(y_test))
    else:
        my_range = int(max(y_predict))
    plt.scatter(range(len(y_test)), y_test, color='blue')
    plt.scatter(range(len(y_predict)), y_predict, color='red')
    plt.title(regressorName)
    plt.show()
    return


plotGraph(y_test, y_predict_Elastic, "Elastic: RED - predicted. BLUE - original")
plotGraph(y_test, y_predict, "Ridge: RED - predicted. BLUE - original")

# Cross validation

In [ ]:
def cross_val(model, X_train, y_train):
  scoring = {'R2': 'r2',
            '-MSE': 'neg_mean_squared_error',
            '-MAE': 'neg_mean_absolute_error',
            'Max': 'max_error'}


  scores = cross_validate(model, X_train, y_train,
                        scoring=scoring, cv=ShuffleSplit(n_splits=5, random_state=42) )

  print('CV Results')
  DF_cv_linreg = pd.DataFrame(scores)
  display(DF_cv_linreg)
  print('\n')
  print(DF_cv_linreg.mean()[2:])
  print('\n')

cross_val(model, X_train, y_train)

# Tests

### Export model

In [ ]:
import pickle

In [ ]:
with open('model.pkl', 'wb') as f:
    pickle.dump(model, f)

In [ ]:
! ls

In [ ]:
import pytest

In [ ]:
! rm /kaggle/working/water-pollution/Tests
! mkdir /kaggle/working/water-pollution/Tests
! touch /kaggle/working/water-pollution/Tests.py

In [ ]:
%%writefile /kaggle/working/water-pollution/Tests.py

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import r2_score
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNetCV
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import GridSearchCV
import pickle


def test_data_preprocessing():
    DF_train = pd.read_csv('/kaggle/input/water-pollution/water_pollution_dataset/Train.csv', delimiter = ',')
    DF_test = pd.read_csv('/kaggle/input/water-pollution/water_pollution_dataset/Test.csv', delimiter = ',')
    Target = pd.read_csv('/kaggle/input/water-pollution/water_pollution_dataset/Target.csv', delimiter = ',')
    Submission = pd.read_csv('/kaggle/input/water-pollution/water_pollution_dataset/Submission.csv', delimiter = ',')
    
    
    df = pd.concat([DF_train, DF_test])
    
    cat_columns = ['code', 'period', 'id', 'Country', 'year']
    num_columns = [ 'tourists', 'venue', 'rate', 'food', 'glass', 'metal', 'other', 'paper', 'plastic', 'leather', 'green_waste', 'waste_recycling'] 
    
    df_ohe = pd.get_dummies(df[cat_columns])
    df_all = pd.concat([df_ohe, df[num_columns]], axis=1)
    
    # Creating StandardScaler Object
    scaler = StandardScaler()
    df_scale = pd.DataFrame(scaler.fit_transform(df_all[num_columns]))
    df_all = pd.concat([df_ohe.reset_index(drop=True), df_scale.reset_index(drop=True)], axis=1)
    
    len_test = len(DF_test)
    train = df_all.iloc[:-len_test]
    test = df_all.iloc[-len_test:]
    target_orig = Target['polution']
    
    X, y = pd.DataFrame(train).values, target_orig.values
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    with open('model.pkl', 'rb') as f:
        loaded_model = pickle.load(f)
    y_predict = loaded_model.predict(X_test)
    assert r2_score(y_test,y_predict) < 0.9
 

def test_model_2():
    DF_train = pd.read_csv('/kaggle/input/water-pollution/water_pollution_dataset/Train.csv', delimiter = ',')
    DF_test = pd.read_csv('/kaggle/input/water-pollution/water_pollution_dataset/Test.csv', delimiter = ',')
    Target = pd.read_csv('/kaggle/input/water-pollution/water_pollution_dataset/Target.csv', delimiter = ',')
    Submission = pd.read_csv('/kaggle/input/water-pollution/water_pollution_dataset/Submission.csv', delimiter = ',')
    
    
    df = pd.concat([DF_train, DF_test])
    
    cat_columns = ['code', 'period', 'id', 'Country', 'year']
    num_columns = [ 'tourists', 'venue', 'rate', 'food', 'glass', 'metal', 'other', 'paper', 'plastic', 'leather', 'green_waste', 'waste_recycling'] 
   
    df_ohe = pd.get_dummies(df[cat_columns])
    df_all = pd.concat([df_ohe, df[num_columns]], axis=1)
    
    # Creating StandardScaler Object
    scaler = StandardScaler()
    df_scale = pd.DataFrame(scaler.fit_transform(df_all[num_columns]))
    df_all = pd.concat([df_ohe.reset_index(drop=True), df_scale.reset_index(drop=True)], axis=1)
    
    len_test = len(DF_test)
    train = df_all.iloc[:-len_test]
    test = df_all.iloc[-len_test:]
    target_orig = Target['polution']
    
    X, y = pd.DataFrame(train).values, target_orig.values
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.9, random_state=42)
    
    with open('model.pkl', 'rb') as f:
        loaded_model = pickle.load(f)
    y_predict = loaded_model.predict(X_test)
    assert r2_score(y_test,y_predict) < 0.4 

In [ ]:
! cat /kaggle/working/water-pollution/Tests.py 

In [ ]:
!pytest /kaggle/working/water-pollution/Tests.py

In [ ]:
y_predict = model.predict(test)

# Convert back from log scaling
#y_predict = np.exp(y_predict)

Submission['polution'] = y_predict
Submission

In [ ]:
! rm /kaggle/working/water-pollution/Submission.csv
! mkdir /kaggle/working/water-pollution/
Submission['polution'][27] = 2.0

In [ ]:
Submission.to_csv('/kaggle/working/water-pollution/Submission.csv',index=False)